# Benchmarks

## Julia

In [15]:
using Random # standard library
Random.seed!(123) # seed

MersenneTwister(123)

In [16]:
x = rand(1_000_000) # 1 million random numbers in [0, 1)

@time sum(x) # first run includes compilation time

  0.001450 seconds (1 allocation: 16 bytes)


500060.34072352527

In [17]:
using BenchmarkTools

bm = @benchmark sum($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     522.901 μs (0.00% GC)
  median time:      529.320 μs (0.00% GC)
  mean time:        554.905 μs (0.00% GC)
  maximum time:     5.327 ms (0.00% GC)
  --------------
  samples:          8716
  evals/sample:     1

In [18]:
using Statistics # standard library
benchmark_result = Dict() # a dictionary to store median runtime (in milliseconds)
benchmark_result["Julia builtin"] = median(bm.times) / 1e6

0.52932

## R

In [19]:
using RCall

In [20]:
R"""
library(microbenchmark)
y <- $x
rbm <- microbenchmark(sum(y))
"""

RObject{VecSxp}
Unit: milliseconds
   expr     min       lq     mean   median       uq      max neval
 sum(y) 2.47414 2.525721 2.784311 2.620521 2.910491 4.683505   100


In [21]:
# store median runtime (in milliseconds)
@rget rbm # dataframe
benchmark_result["R builtin"] = median(rbm[!,:time]) / 1e6

2.6205205

## Python

In [22]:
using PyCall
PyCall.pyversion

v"3.8.8"

In [23]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")
bm = @benchmark $pysum($x)

BenchmarkTools.Trial: 
  memory estimate:  336 bytes
  allocs estimate:  6
  --------------
  minimum time:     84.460 ms (0.00% GC)
  median time:      85.512 ms (0.00% GC)
  mean time:        91.321 ms (0.00% GC)
  maximum time:     153.004 ms (0.00% GC)
  --------------
  samples:          55
  evals/sample:     1

In [24]:
# store median runtime (in miliseconds)
benchmark_result["Python builtin"] = median(bm.times) / 1e6

85.512465

### Python numpy

In [25]:
# bring in sum function from Numpy 
numpy_sum = pyimport("numpy")."sum"

PyObject <function sum at 0x7fbbff18b160>

In [26]:
bm = @benchmark $numpy_sum($x)

BenchmarkTools.Trial: 
  memory estimate:  336 bytes
  allocs estimate:  6
  --------------
  minimum time:     553.477 μs (0.00% GC)
  median time:      586.629 μs (0.00% GC)
  mean time:        620.064 μs (0.00% GC)
  maximum time:     9.070 ms (0.00% GC)
  --------------
  samples:          7814
  evals/sample:     1

In [27]:
# store median runtime (in miliseconds)
benchmark_result["Python numpy"] = median(bm.times) / 1e6

0.586629

## Summary

In [28]:
benchmark_result

Dict{Any, Any} with 4 entries:
  "R builtin"      => 2.62052
  "Julia builtin"  => 0.52932
  "Python builtin" => 85.5125
  "Python numpy"   => 0.586629